In [1]:
import os
import fnmatch
from textblob import TextBlob
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents
import regex as re
import operator
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import pickle
from nltk.corpus import stopwords

In [2]:
path = 'C:/Users/zoe_k/anaconda3/envs/ddb/op_spam_v1.4'

label = []

configfiles = [os.path.join(subdir,f)
for subdir, dirs, files in os.walk(path)
    for f in fnmatch.filter(files, '*.txt')]

In [3]:
len(configfiles)

1600

In [4]:
configfiles[1]

'C:/Users/zoe_k/anaconda3/envs/ddb/op_spam_v1.4\\negative_polarity\\deceptive_from_MTurk\\fold1\\d_hilton_10.txt'

In [5]:
for f in configfiles:
    c = re.search('(trut|deceptiv)\w',f)
    label.append(c.group())

In [6]:
labels = pd.DataFrame(label, columns = ['Labels'])

In [7]:
labels.head(5)

,Labels
0,deceptive
1,deceptive
2,deceptive
3,deceptive
4,deceptive


In [8]:
review = []
directory =os.path.join("C:/Users/zoe_k/anaconda3/envs/ddb/op_spam_v1.4/")
for subdir,dirs ,files in os.walk(directory):
   # print (subdir)
    for file in files:
        if fnmatch.filter(files, '*.txt'):
            f=open(os.path.join(subdir, file),'r')
            a = f.read()
            review.append(a)

In [9]:
reviews = pd.DataFrame(review, columns = ['HotelReviews'])

In [10]:
reviews.head(5)

,HotelReviews
0,We stayed at the Schicago Hilton for 4 days an...
1,Hotel is located 1/2 mile from the train stati...
2,I made my reservation at the Hilton Chicago be...
3,"When most people think Hilton, they think luxu..."
4,My husband and I recently stayed stayed at the...


In [11]:
result = pd.merge(reviews, labels,right_index=True,left_index = True)


result['HotelReviews'] = result['HotelReviews'].map(lambda x: x.lower())

In [12]:
result.head()

,HotelReviews,Labels
0,we stayed at the schicago hilton for 4 days an...,deceptive
1,hotel is located 1/2 mile from the train stati...,deceptive
2,i made my reservation at the hilton chicago be...,deceptive
3,"when most people think hilton, they think luxu...",deceptive
4,my husband and i recently stayed stayed at the...,deceptive


In [23]:
from nltk.corpus import stopwords

In [24]:
stop = stopwords.words('english')

result['review_without_stopwords'] = result['HotelReviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [25]:
result.head()

,HotelReviews,Labels,review_without_stopwords
0,we stayed at the schicago hilton for 4 days an...,deceptive,stayed schicago hilton 4 days 3 nights confere...
1,hotel is located 1/2 mile from the train stati...,deceptive,hotel located 1/2 mile train station quite hik...
2,i made my reservation at the hilton chicago be...,deceptive,made reservation hilton chicago believing goin...
3,"when most people think hilton, they think luxu...",deceptive,"people think hilton, think luxury. know did. w..."
4,my husband and i recently stayed stayed at the...,deceptive,husband recently stayed stayed hilton chicago ...


In [29]:
def pos(review_without_stopwords):
    return TextBlob(review_without_stopwords).tags

In [30]:
os = result.review_without_stopwords.apply(pos)
os1 = pd.DataFrame(os)

In [31]:
os1.head()

,review_without_stopwords
0,"[(stayed, VBN), (schicago, JJ), (hilton, NN), ..."
1,"[(hotel, NN), (located, VBD), (1/2, CD), (mile..."
2,"[(made, VBN), (reservation, NN), (hilton, NN),..."
3,"[(people, NNS), (think, VBP), (hilton, NN), (t..."
4,"[(husband, NN), (recently, RB), (stayed, VBD),..."


In [32]:
os1['pos'] = os1['review_without_stopwords'].map(lambda x:" ".join(["/".join(x) for x in x ]) )

In [33]:
result = result = pd.merge(result, os1,right_index=True,left_index = True)
result.head()

,HotelReviews,Labels,review_without_stopwords_x,review_without_stopwords_y,pos
0,we stayed at the schicago hilton for 4 days an...,deceptive,stayed schicago hilton 4 days 3 nights confere...,"[(stayed, VBN), (schicago, JJ), (hilton, NN), ...",stayed/VBN schicago/JJ hilton/NN 4/CD days/NNS...
1,hotel is located 1/2 mile from the train stati...,deceptive,hotel located 1/2 mile train station quite hik...,"[(hotel, NN), (located, VBD), (1/2, CD), (mile...",hotel/NN located/VBD 1/2/CD mile/NN train/NN s...
2,i made my reservation at the hilton chicago be...,deceptive,made reservation hilton chicago believing goin...,"[(made, VBN), (reservation, NN), (hilton, NN),...",made/VBN reservation/NN hilton/NN chicago/NN b...
3,"when most people think hilton, they think luxu...",deceptive,"people think hilton, think luxury. know did. w...","[(people, NNS), (think, VBP), (hilton, NN), (t...",people/NNS think/VBP hilton/NN think/VBP luxur...
4,my husband and i recently stayed stayed at the...,deceptive,husband recently stayed stayed hilton chicago ...,"[(husband, NN), (recently, RB), (stayed, VBD),...",husband/NN recently/RB stayed/VBD stayed/JJ hi...


In [34]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=13)

In [35]:
tf_vect = TfidfVectorizer(lowercase = True, use_idf=True, smooth_idf=True, sublinear_tf=False)

X_train_tf = tf_vect.fit_transform(review_train)
X_test_tf = tf_vect.transform(review_test)

In [36]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='linear'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    return grid_search.best_params_

In [37]:
svc_param_selection(X_train_tf,label_train,5)

{'C': 1, 'gamma': 0.001}

In [38]:
clf = svm.SVC(C=10,gamma=0.001,kernel='linear')
clf.fit(X_train_tf,label_train)
pred = clf.predict(X_test_tf)

In [39]:
with open('vectorizer.pickle', 'wb') as fin:
    pickle.dump(tf_vect, fin)

In [40]:
with open('mlmodel.pickle','wb') as f:
    pickle.dump(clf,f)

In [41]:
pkl = open('mlmodel.pickle', 'rb')
clf = pickle.load(pkl)   
vec = open('vectorizer.pickle', 'rb')
tf_vect = pickle.load(vec)

In [42]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [43]:
print(metrics.accuracy_score(label_test, pred))

0.890625


In [44]:
print (confusion_matrix(label_test, pred))

[[139  16]
 [ 19 146]]


In [45]:
print (classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.88      0.90      0.89       155
       truth       0.90      0.88      0.89       165

    accuracy                           0.89       320
   macro avg       0.89      0.89      0.89       320
weighted avg       0.89      0.89      0.89       320



In [46]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=1)

In [47]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [48]:
print(metrics.accuracy_score(label_test, pred))

0.984375


In [49]:
print (confusion_matrix(label_test, pred))

[[149   2]
 [  3 166]]


In [50]:
print (classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.98      0.99      0.98       151
       truth       0.99      0.98      0.99       169

    accuracy                           0.98       320
   macro avg       0.98      0.98      0.98       320
weighted avg       0.98      0.98      0.98       320



In [51]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=10)

In [52]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [53]:
print(metrics.accuracy_score(label_test, pred))

0.975


In [54]:
print (confusion_matrix(label_test, pred))

[[157   3]
 [  5 155]]


In [55]:
print (classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.97      0.98      0.98       160
       truth       0.98      0.97      0.97       160

    accuracy                           0.97       320
   macro avg       0.98      0.97      0.97       320
weighted avg       0.98      0.97      0.97       320



In [56]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=42)

In [57]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [58]:
print(metrics.accuracy_score(label_test, pred))

0.978125


In [59]:
print (confusion_matrix(label_test, pred))

[[166   2]
 [  5 147]]


In [60]:
print (classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.97      0.99      0.98       168
       truth       0.99      0.97      0.98       152

    accuracy                           0.98       320
   macro avg       0.98      0.98      0.98       320
weighted avg       0.98      0.98      0.98       320



In [61]:
def test_string(s):
    X_test_tf = tf_vect.transform([s])
    y_predict = clf.predict(X_test_tf)
    return y_predict

In [62]:
test_string("The hotel was good.The room had a 27-inch Samsung led tv, a microwave.The room had a double bed")

array(['truth'], dtype=object)

In [68]:
test_string("we stayed at the schicago hilton for 4 day and it was an amazing experience")

array(['deceptive'], dtype=object)